In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 2


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-06 06:26:35] ax.service.ax_client: Generated new trial 32 with parameters {'Drug': 69, 'SL_1': 8, 'SL_2': 47, 'SL_3': 0, 'LL_1': 39, 'LL_2': 0, 'LL_3': 39, 'P_1': 22, 'P_2': 68, 'P_3': 0, 'S_1': 94, 'S_2': 91, 'S_3': 52, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude 

35 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,94,91,52,0
1,97,92,47,0
2,62,0,42,0
3,29,0,66,0
4,0,0,54,0
5,0,0,100,0
6,0,0,100,0
7,0,0,0,0
8,0,0,0,0
9,100,100,63,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
7            39    97    74     0     0     0     0     0    0    0  ...    0   
8            40    98    73     0     0     0     0     0    0    0  ...    0   
11           43    91    72     0     0     0     0     0    0    0  ...    0   
12           44    96    72     0     0     0     0     0    0    0  ...    0   
13           45   100    73     0     0     0     0     0    0    0  ...    0   
14           46   100    73     0     0     0     0     0    0    0  ...    0   
15           47   100    73     0     0     0     0     0    0    0  ...    0   

    S_2  S_3  Water  Solu Solu_STD  Size Size_STD   PDI PDI_STD  
7     0    0      0  None     None  None     None  None    None  
8     0    0      0  None     None  None     None  None    None  
11    0    0      0  None     None  None     None  None    None  
12    0    0      0  None     None  None     None  None   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,94,91,52,0
1,97,92,47,0
2,62,0,42,0
3,29,0,66,0
4,0,0,54,0
5,0,0,100,0
6,0,0,100,0
7,0,0,0,100
8,0,0,0,100
9,100,100,63,0


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000
mean,39.500000,0.508375,0.300750,0.070937,0.0,0.008375,0.0,0.008375,0.004687,0.098438,0.0,0.170375,0.094812,0.297312,0.437500
std,4.760952,0.117199,0.174065,0.132882,0.0,0.033500,0.0,0.033500,0.018750,0.168405,0.0,0.239513,0.169685,0.398086,0.512348
min,32.000000,0.236000,0.027000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,35.750000,0.459750,0.147250,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,39.500000,0.541000,0.422000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.099500,0.000000
75%,43.250000,0.574250,0.430000,0.040250,0.0,0.000000,0.0,0.000000,0.000000,0.182750,0.0,0.384250,0.090750,0.476750,1.000000
max,47.000000,0.740000,0.495000,0.371000,0.0,0.134000,0.0,0.134000,0.075000,0.550000,0.0,0.637000,0.390000,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.0,16.0000,16.0,16.0000,16.000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000
mean,39.500000,90.562500,52.375000,16.187500,0.0,2.4375,0.0,2.4375,1.375,19.437500,0.0,29.250000,20.750000,32.750000,43.750000
std,4.760952,12.344871,29.197888,29.584272,0.0,9.7500,0.0,9.7500,5.500,30.850108,0.0,42.140242,38.520125,37.199462,51.234754
min,32.000000,62.000000,6.000000,0.000000,0.0,0.0000,0.0,0.0000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,35.750000,89.000000,32.000000,0.000000,0.0,0.0000,0.0,0.0000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,39.500000,96.500000,72.000000,0.000000,0.0,0.0000,0.0,0.0000,0.000,0.000000,0.0,0.000000,0.000000,21.000000,0.000000
75%,43.250000,98.500000,73.000000,11.750000,0.0,0.0000,0.0,0.0000,0.000,44.500000,0.0,68.000000,12.250000,56.250000,100.000000
max,47.000000,100.000000,92.000000,78.000000,0.0,39.0000,0.0,39.0000,22.000,83.000000,0.0,100.000000,100.000000,100.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.000000,16.0,16.00000,16.0,16.00000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000
mean,39.500000,177.931250,105.262500,24.828125,0.0,2.93125,0.0,2.93125,1.640625,34.453125,0.0,170.375000,94.812500,297.312500,437.500000,0.021875
std,4.760952,41.019816,60.922726,46.508748,0.0,11.72500,0.0,11.72500,6.562500,58.941908,0.0,239.513082,169.684892,398.086041,512.347538,0.154886
min,32.000000,82.600000,9.450000,0.000000,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.350000
25%,35.750000,160.912500,51.537500,0.000000,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.500000,189.350000,147.700000,0.000000,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,99.500000,0.000000,0.000000
75%,43.250000,200.987500,150.500000,14.087500,0.0,0.00000,0.0,0.00000,0.000000,63.962500,0.0,384.250000,90.750000,476.750000,1000.000000,0.000000
max,47.000000,259.000000,173.250000,129.850000,0.0,46.90000,0.0,46.90000,26.250000,192.500000,0.0,637.000000,390.000000,1000.000000,1000.000000,0.350000


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       2.8
SL_1       1.7
SL_2       0.4
SL_3       0.0
LL_1       0.0
LL_2       0.0
LL_3       0.0
P_1        0.0
P_2        0.6
P_3        0.0
S_1        2.7
S_2        1.5
S_3        4.8
Water      7.0
Solvent    0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_2_OT_2_protocol_1.py',
 'protocol/iteration_2_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,32,0.236,0.027,0.161,0,0.134,0,0.134,0.075,0.233,0,0.397,0.384,0.219,0,9,0
1,33,0.343,0.049,0.304,0,0.000,0,0.000,0.000,0.304,0,0.411,0.390,0.199,0,6,0
2,34,0.402,0.133,0.299,0,0.000,0,0.000,0.000,0.166,0,0.596,0.000,0.404,0,5,0
3,35,0.476,0.152,0.371,0,0.000,0,0.000,0.000,0.000,0,0.305,0.000,0.695,0,4,0
4,36,0.524,0.476,0.000,0,0.000,0,0.000,0.000,0.000,0,0.000,0.000,1.000,0,2,0
5,37,0.740,0.260,0.000,0,0.000,0,0.000,0.000,0.000,0,0.000,0.000,1.000,0,2,0
6,38,0.505,0.495,0.000,0,0.000,0,0.000,0.000,0.000,0,0.000,0.000,1.000,0,2,0
7,39,0.567,0.433,0.000,0,0.000,0,0.000,0.000,0.000,0,0.000,0.000,0.000,1,1,0
8,40,0.573,0.427,0.000,0,0.000,0,0.000,0.000,0.000,0,0.000,0.000,0.000,1,1,0
9,41,0.411,0.040,0.000,0,0.000,0,0.000,0.000,0.550,0,0.380,0.380,0.240,0,5,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,164.895112,0.181434
1,Good,A2,157.542098,0.227841
2,Good,A3,161.371982,0.216412
3,Good,A4,141.103875,0.461009
4,Good,A5,144.668398,0.441006
5,Good,A6,141.469493,0.248919
6,Good,B1,232.130009,0.180113
7,Good,B2,222.294052,0.217373
8,Good,B3,239.587270,0.264612
9,Good,B4,292.851735,0.213340


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,161.269731,3.677573,0.208562,0.024179,1
1,Formulation 2,142.413922,1.960974,0.383645,0.117104,1
2,Formulation 3,231.337111,8.673832,0.220699,0.042348,1
3,Formulation 4,298.411876,7.863228,0.234908,0.030502,1
4,Formulation 5,247.174906,9.397340,0.262497,0.008046,1
5,Formulation 6,215.975989,10.648979,0.332667,0.023294,1
6,Formulation 7,246.727760,8.389961,0.352208,0.060680,1
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,240.738586,4.893345,0.266360,0.083756,1
9,Formulation 10,28.206659,0.629233,0.149506,0.034986,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,32,0.236,0.027,0.161,0,0.134,0,0.134,0.075,0.233,...,0.219,0,9,0,Formulation 1,161.269731,3.677573,0.208562,0.024179,1
1,33,0.343,0.049,0.304,0,0.000,0,0.000,0.000,0.304,...,0.199,0,6,0,Formulation 2,142.413922,1.960974,0.383645,0.117104,1
2,34,0.402,0.133,0.299,0,0.000,0,0.000,0.000,0.166,...,0.404,0,5,0,Formulation 3,231.337111,8.673832,0.220699,0.042348,1
3,35,0.476,0.152,0.371,0,0.000,0,0.000,0.000,0.000,...,0.695,0,4,0,Formulation 4,298.411876,7.863228,0.234908,0.030502,1
4,36,0.524,0.476,0.000,0,0.000,0,0.000,0.000,0.000,...,1.000,0,2,0,Formulation 5,247.174906,9.397340,0.262497,0.008046,1
5,37,0.740,0.260,0.000,0,0.000,0,0.000,0.000,0.000,...,1.000,0,2,0,Formulation 6,215.975989,10.648979,0.332667,0.023294,1
6,38,0.505,0.495,0.000,0,0.000,0,0.000,0.000,0.000,...,1.000,0,2,0,Formulation 7,246.727760,8.389961,0.352208,0.060680,1
7,39,0.567,0.433,0.000,0,0.000,0,0.000,0.000,0.000,...,0.000,1,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,40,0.573,0.427,0.000,0,0.000,0,0.000,0.000,0.000,...,0.000,1,1,0,Formulation 9,240.738586,4.893345,0.266360,0.083756,1
9,41,0.411,0.040,0.000,0,0.000,0,0.000,0.000,0.550,...,0.240,0,5,0,Formulation 10,28.206659,0.629233,0.149506,0.034986,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,0.236,0.027,0.161,0,0.134,0,0.134,0.075,0.233,...,9,0,Formulation 1,161.269731,3.677573,0.208562,0.024179,1,472.0,0
1,33,0.343,0.049,0.304,0,0.000,0,0.000,0.000,0.304,...,6,0,Formulation 2,142.413922,1.960974,0.383645,0.117104,1,686.0,0
2,34,0.402,0.133,0.299,0,0.000,0,0.000,0.000,0.166,...,5,0,Formulation 3,231.337111,8.673832,0.220699,0.042348,1,804.0,0
3,35,0.476,0.152,0.371,0,0.000,0,0.000,0.000,0.000,...,4,0,Formulation 4,298.411876,7.863228,0.234908,0.030502,1,952.0,0
4,36,0.524,0.476,0.000,0,0.000,0,0.000,0.000,0.000,...,2,0,Formulation 5,247.174906,9.397340,0.262497,0.008046,1,1048.0,0
5,37,0.740,0.260,0.000,0,0.000,0,0.000,0.000,0.000,...,2,0,Formulation 6,215.975989,10.648979,0.332667,0.023294,1,1480.0,0
6,38,0.505,0.495,0.000,0,0.000,0,0.000,0.000,0.000,...,2,0,Formulation 7,246.727760,8.389961,0.352208,0.060680,1,1010.0,0
7,39,0.567,0.433,0.000,0,0.000,0,0.000,0.000,0.000,...,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,40,0.573,0.427,0.000,0,0.000,0,0.000,0.000,0.000,...,1,0,Formulation 9,240.738586,4.893345,0.266360,0.083756,1,1146.0,0
9,41,0.411,0.040,0.000,0,0.000,0,0.000,0.000,0.550,...,5,0,Formulation 10,28.206659,0.629233,0.149506,0.034986,1,822.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.161270,0.003678,0.236,0.0,0.208562,0.024179,0.750000
1,1,0.142414,0.001961,0.343,0.0,0.383645,0.117104,0.500000
2,1,0.231337,0.008674,0.402,0.0,0.220699,0.042348,0.416667
3,1,0.298412,0.007863,0.476,0.0,0.234908,0.030502,0.333333
4,1,0.247175,0.009397,0.524,0.0,0.262497,0.008046,0.166667
5,1,0.215976,0.010649,0.740,0.0,0.332667,0.023294,0.166667
6,1,0.246728,0.008390,0.505,0.0,0.352208,0.060680,0.166667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.240739,0.004893,0.573,0.0,0.266360,0.083756,0.083333
9,1,0.028207,0.000629,0.411,0.0,0.149506,0.034986,0.416667


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.161270,0.003678,0.236,0.0,0.208562,0.024179,0.750000
1,1,0.142414,0.001961,0.343,0.0,0.383645,0.117104,0.500000
2,1,0.231337,0.008674,0.402,0.0,0.220699,0.042348,0.416667
3,1,0.298412,0.007863,0.476,0.0,0.234908,0.030502,0.333333
4,1,0.247175,0.009397,0.524,0.0,0.262497,0.008046,0.166667
5,1,0.215976,0.010649,0.740,0.0,0.332667,0.023294,0.166667
6,1,0.246728,0.008390,0.505,0.0,0.352208,0.060680,0.166667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.240739,0.004893,0.573,0.0,0.266360,0.083756,0.083333
9,1,0.028207,0.000629,0.411,0.0,0.149506,0.034986,0.416667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,0.236,0.027,0.161,0,0.134,0,0.134,0.075,0.233,...,0.750000,0,Formulation 1,0.161270,0.003678,0.208562,0.024179,1,0.236,0
1,33,0.343,0.049,0.304,0,0.000,0,0.000,0.000,0.304,...,0.500000,0,Formulation 2,0.142414,0.001961,0.383645,0.117104,1,0.343,0
2,34,0.402,0.133,0.299,0,0.000,0,0.000,0.000,0.166,...,0.416667,0,Formulation 3,0.231337,0.008674,0.220699,0.042348,1,0.402,0
3,35,0.476,0.152,0.371,0,0.000,0,0.000,0.000,0.000,...,0.333333,0,Formulation 4,0.298412,0.007863,0.234908,0.030502,1,0.476,0
4,36,0.524,0.476,0.000,0,0.000,0,0.000,0.000,0.000,...,0.166667,0,Formulation 5,0.247175,0.009397,0.262497,0.008046,1,0.524,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 32 with data: {'Solu': (0.236, 0), 'Size': (0.16127, 0.003678), 'PDI': (0.208562, 0.024179)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 33 with data: {'Solu': (0.343, 0), 'Size': (0.142414, 0.001961), 'PDI': (0.383645, 0.117104)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 34 with data: {'Solu': (0.402, 0), 'Size': (0.231337, 0.008674), 'PDI': (0.220699, 0.042348)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 35 with data: {'Solu': (0.476, 0), 'Size': (0.298412, 0.007863), 'PDI': (0.234908, 0.030502)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 36 with data: {'Solu': (0.524, 0), 'Size': (0.247175, 0.009397), 'PDI': (0.262497, 0.008046)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 37 with data: {'Solu': (0.74, 0), 'Size': (0.215976, 0.010649), 'PDI': (0.332667, 0.023294)}.
[INFO 08-06 08:49:02] ax.service.ax_client: Completed trial 38 with data